In [3]:
%pip install pypdf2

Note: you may need to restart the kernel to use updated packages.


### Carregando e extraindo dados

In [19]:
import re
import os
import random
from collections import Counter
import unicodedata

# --- Tente com PyPDF2 ---
try:
    import PyPDF2
    pdf_reader_type = 'PyPDF2'
except ImportError:
    print("PyPDF2 não encontrado. Tentando PyMuPDF...")
    # --- Tente com PyMuPDF (fitz) ---
    try:
        import fitz # PyMuPDF
        pdf_reader_type = 'PyMuPDF'
    except ImportError:
        print("Erro: Nem PyPDF2 nem PyMuPDF (fitz) estão instalados.")
        print("Execute 'pip install PyPDF2' ou 'pip install PyMuPDF'")
        pdf_reader_type = None

# --- Configurações ---
caminho_arquivo_pdf = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/Carvalho_1987_DicTupiAntigo-Port_OCR.pdf' # <<< COLOQUE O CAMINHO CORRETO AQUI

caminho_arquivo_txt_extraido = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_bruto_extraido.txt' # Arquivo para salvar o texto bruto extraído
caminho_arquivo_limpo_fase1 = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_bruto_limpo.txtfase1.txt'

# --- Extrair Texto do PDF ---
texto_original = ""
linhas_originais = []

if pdf_reader_type and os.path.exists(caminho_arquivo_pdf):
    print(f"Tentando ler '{caminho_arquivo_pdf}' usando {pdf_reader_type}...")
    try:
        if pdf_reader_type == 'PyPDF2':
            with open(caminho_arquivo_pdf, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                num_paginas = len(pdf_reader.pages)
                print(f"PDF tem {num_paginas} páginas.")
                for page_num in range(num_paginas):
                    if (page_num + 1) % 50 == 0: # Feedback a cada 50 páginas
                         print(f"Processando página {page_num + 1}/{num_paginas}...")
                    page = pdf_reader.pages[page_num]
                    texto_original += page.extract_text() + "\n" # Adiciona nova linha entre páginas
            print("Extração com PyPDF2 concluída.")

        elif pdf_reader_type == 'PyMuPDF':
            doc = fitz.open(caminho_arquivo_pdf)
            num_paginas = doc.page_count
            print(f"PDF tem {num_paginas} páginas.")
            for page_num in range(num_paginas):
                 if (page_num + 1) % 50 == 0: # Feedback a cada 50 páginas
                         print(f"Processando página {page_num + 1}/{num_paginas}...")
                 page = doc.load_page(page_num)
                 texto_original += page.get_text("text") + "\n" # Adiciona nova linha entre páginas
            doc.close()
            print("Extração com PyMuPDF concluída.")

        # Salvar o texto bruto extraído (bom para referência)
        with open(caminho_arquivo_txt_extraido, 'w', encoding='utf-8') as f_out:
             f_out.write(texto_original)
        print(f"Texto bruto extraído salvo em '{caminho_arquivo_txt_extraido}'")

        # Dividir o texto em linhas para as próximas etapas
        linhas_originais = texto_original.splitlines()
        print(f"Texto carregado: {len(texto_original)} caracteres, {len(linhas_originais)} linhas (aprox.)")

    except Exception as e:
        print(f"Erro ao processar o PDF: {e}")
        texto_original = None
        linhas_originais = []

elif not pdf_reader_type:
    print("Nenhuma biblioteca de leitura de PDF funcional encontrada.")
else:
     print(f"Erro: Arquivo PDF não encontrado em '{caminho_arquivo_pdf}'")

# --- Verificação Simples ---
if linhas_originais:
    print("\n--- Primeiras 5 Linhas Extraídas ---")
    for i, linha in enumerate(linhas_originais[:5]):
        print(f"{i:03d}: {linha}")
else:
    print("\nNão foi possível carregar o texto do PDF.")

# (Opcional: Se a extração falhar, você pode tentar copiar manualmente
#  o texto de algumas páginas do PDF para um arquivo .txt e usar o
#  código da resposta anterior para carregar o .txt)
# print("\nAlternativa: Copie o texto manualmente do PDF para um arquivo .txt")
# print("E ajuste 'caminho_arquivo_ocr' na célula original para ler esse .txt")

Erro: Arquivo PDF não encontrado em 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/Carvalho_1987_DicTupiAntigo-Port_OCR.pdf'

Não foi possível carregar o texto do PDF.


### Normalizando

In [11]:
import re
import json
import os
import pandas as pd # Import pandas for the validation step later
import random     # Import random for sampling

# ==============================================================================
# --- Configurações ---
# ==============================================================================

# Caminho para o SEU arquivo bruto extraido do PDF
# AJUSTE ESTE CAMINHO CONFORME NECESSÁRIO
caminho_arquivo_bruto = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_bruto_extraido.txt'

# Caminho para salvar o texto pré-filtrado (OPCIONAL, para debug)
# Se não quiser salvar, pode deixar comentado ou None
caminho_arquivo_pre_filtrado_opcional = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_pre_filtrado_debug.txt'
# caminho_arquivo_pre_filtrado_opcional = None # Descomente esta linha para não salvar

# Caminho para salvar os dados estruturados finais em JSON - SAÍDA PRINCIPAL
# Usando v2 para diferenciar desta versão do script
caminho_arquivo_json = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dicionario_estruturado_final_v2.json'

# Caminho para salvar informações sobre entradas que causaram erros
caminho_arquivo_erros = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/entradas_com_erro_v2.txt'

# ==============================================================================
# --- PASSO 1: Carregar o Texto Bruto ---
# ==============================================================================
print("--- PASSO 1: Carregando Texto Bruto ---")
texto_completo_bruto = None
encodings_to_try = ['utf-8', 'latin-1', 'cp1252'] # Lista de codificações comuns
for enc in encodings_to_try:
    try:
        with open(caminho_arquivo_bruto, 'r', encoding=enc) as f:
            texto_completo_bruto = f.read()
        print(f"Texto bruto carregado com sucesso de '{caminho_arquivo_bruto}' usando encoding '{enc}'.")
        print(f"Tamanho: {len(texto_completo_bruto)} caracteres.")
        break # Sai do loop se carregar com sucesso
    except FileNotFoundError:
        print(f"Erro Crítico: Arquivo bruto não encontrado em '{caminho_arquivo_bruto}'")
        texto_completo_bruto = None
        break
    except Exception as e:
        print(f"Falha ao ler com encoding '{enc}': {e}")
        continue # Tenta o próximo encoding

if texto_completo_bruto is None:
    print("Erro Crítico: Não foi possível carregar o arquivo bruto. Verifique o caminho e a codificação.")
    # Encerra o script ou lida com o erro como preferir
    exit()

# ==============================================================================
# --- PASSO 2: Pré-filtragem em Memória ---
# ==============================================================================
print("\n--- PASSO 2: Iniciando Pré-filtragem em Memória ---")
texto_filtrado_memoria = ""
linhas_removidas_count = 0
if texto_completo_bruto:
    linhas_brutas = texto_completo_bruto.splitlines()
    linhas_filtradas = []
    num_linhas_brutas = len(linhas_brutas)

    for i, linha in enumerate(linhas_brutas):
        linha_strip = linha.strip()

        # Remover linhas vazias
        if not linha_strip:
            linhas_removidas_count += 1
            continue
        # Remover linhas que são apenas números (provavelmente páginas)
        if re.fullmatch(r'\d+', linha_strip):
            linhas_removidas_count += 1
            continue
        # Remover linhas separadoras comuns (estrelas, 'k', traços repetidos)
        if re.fullmatch(r'[\*k -]+', linha_strip) and len(linha_strip) > 3:
             linhas_removidas_count += 1
             continue
        # Remover running headers específicos (Ex: VERBETE / VERBETE?) - AJUSTE SE NECESSÁRIO!
        # Esta regex pode precisar ser adaptada ao padrão exato do seu PDF
        # A heurística verifica se a linha se parece com um header e se está isolada (linhas acima/abaixo curtas ou vazias)
        if i > 0 and i < num_linhas_brutas - 1: # Evita checar primeira/última linha
             if re.fullmatch(r'[A-ZÁÉÍÓÚÇÑŸ\-]+\s*(?:/\s*[A-ZÁÉÍÓÚÇÑŸ\-]+\??|-)\s*', linha_strip, re.IGNORECASE):
                 # Verifica contexto: linha acima ou abaixo vazia/curta aumenta chance de ser header
                 linha_anterior_vazia = not linhas_brutas[i-1].strip()
                 linha_posterior_vazia = not linhas_brutas[i+1].strip()
                 if linha_anterior_vazia or linha_posterior_vazia or len(linhas_brutas[i-1]) < 15 or len(linhas_brutas[i+1]) < 15:
                    # print(f"Removendo linha de header provável: '{linha_strip}'") # Descomentar para debug
                    linhas_removidas_count += 1
                    continue
        # Remover linhas de referência da biblioteca digital (ajuste se necessário)
        if "Biblioteca Digital Curt Nimuendajú" in linha or "http://www.etnolinguistica.org" in linha:
            linhas_removidas_count += 1
            continue

        # Se passou por todos os filtros, mantém a linha original
        linhas_filtradas.append(linha)

    texto_filtrado_memoria = "\n".join(linhas_filtradas)
    print(f"Pré-filtragem concluída. {linhas_removidas_count} linhas removidas.")
    print(f"Tamanho do texto filtrado: {len(texto_filtrado_memoria)} caracteres.")

    # --- OPCIONAL: Salvar o texto pré-filtrado para debug ---
    if caminho_arquivo_pre_filtrado_opcional:
        try:
            with open(caminho_arquivo_pre_filtrado_opcional, 'w', encoding='utf-8') as f_out:
                f_out.write(texto_filtrado_memoria)
            print(f"Texto pré-filtrado (para debug) salvo em '{caminho_arquivo_pre_filtrado_opcional}'")
        except Exception as e:
            print(f"Erro ao salvar arquivo pré-filtrado opcional: {e}")
    # ---------------------------------------------------------
else:
    print("Texto bruto não carregado, pulando etapas subsequentes.")


# ==============================================================================
# --- PASSO 3: Dividir o Texto Filtrado em Entradas ---
# ==============================================================================
print("\n--- PASSO 3: Dividindo Texto em Entradas ---")
entradas_brutas = []
if texto_filtrado_memoria:
    # Regex para identificar inícios de entrada (mais robusta)
    # Procura linhas que começam com pouca indentação E:
    #   Opção 1: Opcional '— ' seguido por Maiúscula ou Acentuada Maiúscula
    #   Opção 2: Apenas uma Letra Maiúscula (marcador de seção como A, B, C)
    regex_inicio_entrada = re.compile(r"^(?:\s{0,3}(?:—\s)?(?:[A-ZÁÉÍÓÚÇÑŸ]|[ÁÉÍÓÚ]\b))|^(?:\s{0,3}[A-Z]\s*)$", re.MULTILINE)

    indices_inicio = [m.start() for m in regex_inicio_entrada.finditer(texto_filtrado_memoria)]

    if indices_inicio:
        print(f"Encontrados {len(indices_inicio)} potenciais inícios de entrada no texto filtrado.")
        for i in range(len(indices_inicio)):
            start = indices_inicio[i]
            # O fim é o início da próxima entrada, ou o fim do texto
            end = indices_inicio[i+1] if (i+1) < len(indices_inicio) else len(texto_filtrado_memoria)
            bloco = texto_filtrado_memoria[start:end].strip()
            if bloco: # Adiciona apenas se não for vazio
                entradas_brutas.append(bloco)

        print(f"Dividido em {len(entradas_brutas)} entradas brutas.")

        # --- DEBUG: Mostrar algumas entradas brutas ---
        if entradas_brutas:
            print("\n--- Exemplo das primeiras 3 entradas brutas (Após Divisão) ---")
            for i, entrada in enumerate(entradas_brutas[:3]):
                print(f"--- Entrada Bruta {i+1} ---")
                print(entrada) # Mostra a entrada completa para análise
                print("-" * 20)
        # ---------------------------------------------
    else:
        print("Nenhum início de entrada encontrado com o padrão atual no texto filtrado.")
else:
    print("Texto filtrado em memória está vazio.")


# ==============================================================================
# --- PASSO 4: Função de Limpeza Robusta ---
# ==============================================================================
print("\n--- PASSO 4: Definindo Função de Limpeza ---")

def limpar_texto_robusto(texto):
    """
    Aplica um conjunto extensivo de regras de limpeza e normalização ao texto.
    !!!!! ESTA FUNÇÃO PRECISA SER MUITO EXPANDIDA COM REGRAS ESPECÍFICAS !!!!!
    !!!!! BASEADAS NOS ERROS REAIS DO SEU ARQUIVO OCR.             !!!!!
    """
    if not texto: return ""

    # 1. Normalização básica inicial (ex: múltiplos espaços)
    texto = re.sub(r'\s+', ' ', texto).strip()

    # 2. Correções específicas de OCR e Inconsistências (ADICIONE MUITAS MAIS!)
    #    Use um dicionário para facilitar a manutenção. Chaves são os erros, valores as correções.
    #    Seja o mais específico possível para evitar substituições indesejadas.
    correcoes = {
        # Exemplos de erros comuns (Adapte e EXPANDA MUITO!)
         'Prefi.xo': 'Prefixo', 'fiubstantivo': 'Substantivo', 'Substarítivo': 'Substantivo',
         'S)ibsta\'ntivo': 'Substantivo', 'Substantlvo': 'Substantivo', 'Sübstantivo': 'Substantivo',
         'Adj^fcivo': 'Adjetivo', 'A_d jetivo':'Adjetivo', 'Mjetivao': 'Adjetivo',
         'Partlculas': 'Partículas', 'Patticulas': 'Partículas', 'Partícúla': 'Partícula', 'Particlpio':'Particípio',
         'Advérbiorse': 'Advérbio: se','Adverbio:': 'Advérbio:', 'Adverbio': 'Advérbio',
         'Verbotransitivo': 'Verbo transitivo', 'Verbo intransitivo:o': 'Verbo intransitivo:',
         'Verbo paredioativo': 'Verbo predicativo',
         'observagoes': 'observações', 'índío': 'índio', 'nao': 'não', 'Nao': 'Não', 'náo': 'não',
         'fungáo': 'função', 'conjugagao': 'conjugação', 'Cabega': 'Cabeça', 'cabeqa': 'cabeça',
         'composigao': 'composição', 'posipoes': 'posições', 'só ha': 'só há', 'ha': 'há',
         'intradutlvel': 'intradutível', 'próxi mo': 'próximo', 'aqáo': 'ação', 'signíficado': 'significado',
         'á': 'ã', 'Á': 'Ã', '§': 'ç', '£': 'S', # Suposições para símbolos estranhos
         ' 9 ': ' ç ', '9 ': 'ç ', ' 9': ' ç', '(t, 9 )':'(t, ç)', # Correção mais agressiva para 9
         'ü': 'u', 'Ü': 'U', # Normalizar trema se desnecessário
         # 'y': 'y', # Manter Y com trema se for caractere válido no Tupi Antigo do dicionário
         ' l ': ' i ', # l isolado vira i? Cautela.
         'permiss^L': 'permissivo', '^fruto':'1 fruto', ' z ': ' 2 ', 'zGlande':'2 Glande','^contradizer':'1 contradizer', # Tenta numerar
         'si nal': 'sinal', 'planta (desenho)': 'planta (desenho)',
         ' prepoderante':' preponderante', ' oontrário':' contrário', ' riiinca':' nunca',
         ' afcsolutamente':' absolutamente', ' a/grupadas':' agrupadas', ' sicj':' sic ',
         'multidao': 'multidão', 'duplicagao': 'duplicação', 'duplicaqao': 'duplicação',
         'procedéncia': 'procedência', 'género': 'gênero', 'famllia': 'família', 'séxo': 'sexo',
         'mé' : 'mãe', 'Índío': 'índio', 'fíubstantivo':'Substantivo','fago':'fago','recordagao':'recordação',
         'traduqáo': 'tradução', 'conjungáp': 'conjunção', 'Erigado':'Eriçado',
         'diminutl:vo':'diminutivo','répleto':'repleto','pélos':'pelos','respiraqao':'respiração',
         'fiaqao':'fiação','recepgáo':'recepção','AgÉ-AgEMA':'AGÉ-AGEMA', 'las.':'1as.',
         'preposi^ gao':'preposição', 'pessbal':'pessoal','prepos_i gáo':'preposição', 'Adáo':'Adão',
         'abundáncia':'abundância','Preposdgáo':'Preposição','desen^Dlvem':'desenvolvem',
         'mao':'mão','cansago':'cansaço','conjugáo':'conjunção', 'oragáo':'oração', 'prSprio':'próprio',
         'oraqoes':'orações','ünica':'única','preposiqoes':'preposições','preposiqáo':'preposição',
         'entá o':'então','naqu_i':'naquilo','interroga tiva':'interrogativa', 'encru2ilhada':'encruzilhada',
         'fungoes':'funções','sé':'só','agao':'ação','posiqáo':'posição','explrlito':'espírito',
         'relaqao':'relação', 'engros- sar':'engrossar','solugáo':'solução',
         'ÍABA': 'ABA', 'f ': '† ', '+ ': '‡ ', # Usar símbolos distintos ou remover depois
         'tupi) . É': 'tupi). É', # Corrigir espaço antes de ponto final
         '(XE).': '(XE)', '(T-).': '(T-)', '(Q-).': '(Q-)', # Remover ponto após marcadores
         # ... Adicione CENTENAS de regras aqui conforme identificar erros ...
         'excrémenlo': 'excremento', 'portu- gués': 'português', 'porem': 'porém',
         'sé': 'se', 'ré': 'ré', 'é': 'é', 'í': 'i', 'Ó': 'ó', 'ó': 'ó', # Normalizar acentos se necessário
         'qüera':'güera', 'sübstantivo':'substantivo', 'egé':'eçé', # Exemplos de normalização ortográfica Tupi? (Verificar!)
    }

    texto_processado = texto # Começa com o texto já com espaços normalizados

    # Aplica as correções específicas
    # Usar replace simples pode ser mais rápido para muitas regras
    for erro, correcao in correcoes.items():
        texto_processado = texto_processado.replace(erro, correcao)

    # 3. Correções com Regex (para padrões mais complexos)
    # Ex: Corrigir '9' que sobrou dentro de palavras (se necessário e seguro)
    # texto_processado = re.sub(r'(\w)9(\w)', r'\1ç\2', texto_processado)
    # Ex: Normalizar hífens (garantir espaço antes/depois ou remover espaços)
    texto_processado = re.sub(r'\s+-\s+', '-', texto_processado) # Junta hífens soltos
    # Ex: Corrigir espaços antes/depois de parênteses/colchetes/chaves
    texto_processado = re.sub(r'\s*([({\[])\s*', r' \1', texto_processado) # Espaço ANTES de abrir
    texto_processado = re.sub(r'\s*([)}\]])\s*', r'\1 ', texto_processado) # Espaço DEPOIS de fechar

    # 4. Normalização final de Espaçamento e Pontuação
    texto_processado = re.sub(r' +', ' ', texto_processado).strip() # Remove múltiplos espaços
    # Garantir espaço APÓS pontuação comum (exceto fim de string)
    texto_processado = re.sub(r'([.,;:?!])([a-zA-ZÀ-ú0-9])', r'\1 \2', texto_processado)
    # Remover espaço ANTES de pontuação comum
    texto_processado = re.sub(r'\s([.,;:?!])', r'\1', texto_processado)
    # Remover espaço antes de dois pontos, se seguido por espaço (Ex: "Substantivo : algo")
    texto_processado = re.sub(r'\s+:\s+', ': ', texto_processado)
    # Garantir que ':' no fim de classes gramaticais tenha espaço depois se seguido por letra
    texto_processado = re.sub(r'([a-zA-Z]):([a-zA-Z])', r'\1: \2', texto_processado)

    return texto_processado.strip()

print("Função de limpeza definida.")


# ==============================================================================
# --- PASSO 5: Processamento e Extração Refinados ---
# ==============================================================================
dados_estruturados = []
entradas_com_erro = []

# Lista expandida de possíveis inícios de classes gramaticais (terminando com :)
# Adapte esta lista conforme as classes EXATAS do seu dicionário após limpeza
CLASSES_GRAMATICAIS = sorted([ # Ordenar facilita a leitura
    'Prefixo', 'Sufixo verbal', 'Sufixo', 'Afixo', 'Substantivo', 'Gerúndio',
    'Verbo transitivo irregular', 'Verbo intransitivo irregular', # Mais específicos primeiro
    'Verbo transitivo relativo', 'Verbo transitivo', 'Verbo intransitivo',
    'Verbo predicativo', 'Verbo factivo', 'Verbo pronominal', 'Verbo reflexivo',
    'Verbo iterativo', 'Verbo defectivo', 'Verbo', # Genérico
    'Vocativo', 'Demonstrativo', 'Partículas agrupadas', 'Partícula', 'Advérbio',
    'Adjetivo', 'Pronome interrogativo', 'Pronome pessoal', 'Pronome relativo',
    'Pronome indefinido', 'Pronome possessivo', 'Pronome', # Genérico
    'Numeral', 'Interjeigáo', 'Conjunqáo', 'Preposiqáo', 'Locuqáo prepositiva',
    'Locuqáo adverbial', 'Locuqáo substantiva', 'Particípio', 'Ictiologia',
    'Ornitologia', 'Zoologia', 'Botánica', 'Entomologia', 'Conquiliologia',
    'Crustaceologia', 'Astronomia', 'Mineralogia', 'Mitologia', 'Malacologia', # Adicionar mais se houver
    'Fitologia' # Exemplo adicional
], key=len, reverse=True) # Processar nomes mais longos primeiro para evitar matches parciais

# Regex para encontrar uma dessas classes (case-insensitive) seguida por ':' ou fim de string
regex_classe = re.compile(r'(\b(?:' + '|'.join(re.escape(c) for c in CLASSES_GRAMATICAIS) + r')(?:\s+[a-z]+)*)(?::|\s*$)', re.IGNORECASE)

if 'entradas_brutas' in locals() and entradas_brutas:
    print(f"\n--- PASSO 5: Iniciando Processamento e Extração de {len(entradas_brutas)} Entradas ---")
    for i, entrada_bruta in enumerate(entradas_brutas):
        if (i + 1) % 200 == 0:
            print(f"Processando entrada {i+1}/{len(entradas_brutas)}...")

        try:
            entrada_limpa = limpar_texto_robusto(entrada_bruta) # Limpa a entrada inteira primeiro
            linhas_entrada = entrada_limpa.splitlines() # Re-split após limpeza para pegar a primeira linha
            if not linhas_entrada: linhas_limpas = [entrada_limpa] # Caso a limpeza junte tudo

            primeira_linha = linhas_limpas[0].strip()
            corpo_linhas_texto = " ".join(linhas_limpas[1:]).strip() # Junta o resto com espaços

            verbete_tupi = ""
            marcadores = None
            classe_gramatical = None
            texto_para_definicoes = ""
            linha_processada = primeira_linha # Linha para extrair verbete/classe

            # 1. Tenta extrair marcador (primeiro para simplificar a linha)
            match_marcador = re.search(r'(\s*[({\[].*?[)}\]])', linha_processada)
            if match_marcador:
                marcadores = match_marcador.group(1).strip()
                linha_processada = (linha_processada[:match_marcador.start()] + linha_processada[match_marcador.end():]).strip()

            # 2. Tenta encontrar a CLASSE GRAMATICAL na linha processada
            match_classe_obj = regex_classe.search(linha_processada)
            pos_classe = match_classe_obj.start() if match_classe_obj else -1

            if pos_classe != -1:
                 # Classe encontrada na primeira linha
                 verbete_tupi = linha_processada[:pos_classe].strip()
                 classe_achada = match_classe_obj.group(1).strip()
                 # Verifica se a classe achada está na nossa lista (evita falsos positivos)
                 if any(classe_achada.lower() == c.lower() for c in CLASSES_GRAMATICAIS):
                      classe_gramatical = classe_achada
                      texto_para_definicoes = linha_processada[match_classe_obj.end():].strip() + " " + corpo_linhas_texto
                 else: # Falso positivo, tratar como parte do verbete ou definição
                      verbete_tupi = linha_processada.strip() # Assume verbete é tudo
                      texto_para_definicoes = corpo_linhas_texto
            else:
                 # Nenhuma classe na primeira linha, verbete é a linha toda (limpa)
                 verbete_tupi = linha_processada.strip()
                 texto_para_definicoes = corpo_linhas_texto # Definição começa nas linhas seguintes
                 # Tenta achar classe no INÍCIO do corpo
                 match_classe_corpo = regex_classe.match(texto_para_definicoes)
                 if match_classe_corpo:
                      classe_achada_corpo = match_classe_corpo.group(1).strip()
                      if any(classe_achada_corpo.lower() == c.lower() for c in CLASSES_GRAMATICAIS):
                          classe_gramatical = classe_achada_corpo
                          texto_para_definicoes = texto_para_definicoes[match_classe_corpo.end():].strip()

            # 3. Limpeza final do verbete
            verbete_tupi = re.sub(r'[:.—?]+\s*$', '', verbete_tupi).strip() # Remove pontuação final
            if verbete_tupi.startswith('— '): verbete_tupi = verbete_tupi[2:]
            # Evitar remover traço se for parte do nome como AAN-GATU
            # if verbete_tupi.endswith(' —') and not '-' in verbete_tupi[:-2]: verbete_tupi = verbete_tupi[:-2]

            # Normalizar classe gramatical (ex: colocar primeira letra maiúscula)
            if classe_gramatical:
                 classe_gramatical = classe_gramatical.strip().capitalize()


            # 4. Extrair Definições, Exemplos, Referências do texto_para_definicoes
            definicoes = []
            exemplos = []
            referencias = []

            # Dividir o texto por marcadores de início (números, EX:, =, †, ‡)
            # Usando lookahead positivo (?=...) para manter os delimitadores no início das partes
            # O padrão agora é mais explícito
            partes_corpo = re.split(r'(?=\s*(?:[1-9]+\.|†|‡|\^|z\b|EX:| = ))', texto_para_definicoes)

            item_atual = {"tipo": "definicao", "texto": ""} # Começa assumindo definição

            # Processa a primeira parte separadamente se não começar com marcador
            if partes_corpo and partes_corpo[0].strip() and not re.match(r'^\s*(?:[1-9]+\.|†|‡|\^|z\b|EX:| = )', partes_corpo[0]):
                 item_atual["texto"] = partes_corpo[0].strip()
                 partes_corpo = partes_corpo[1:] # Processa o resto a partir do segundo elemento

            for parte in partes_corpo:
                parte = parte.strip()
                if not parte: continue

                match_ex = re.match(r'^\s*EX:\s*(.*)', parte, re.IGNORECASE)
                match_ref = re.match(r'^\s*=\s*(.*)', parte)
                match_num = re.match(r'^\s*([1-9]+\.|†|‡|\^|z\b)\s*(.*)', parte) # Captura o marcador e o texto

                tipo_novo_item = None
                texto_novo_item = ""

                if match_ex:
                     tipo_novo_item = "exemplo"
                     texto_novo_item = match_ex.group(1).strip()
                elif match_ref:
                     tipo_novo_item = "referencia"
                     texto_novo_item = match_ref.group(1).strip()
                elif match_num:
                     tipo_novo_item = "definicao"
                     texto_novo_item = match_num.group(2).strip() # Pega só o texto após o marcador
                else:
                     # Se a parte não começa com marcador conhecido, é continuação do item anterior
                     item_atual["texto"] += " " + parte
                     continue # Pula para a próxima parte sem salvar/resetar item_atual

                # Se encontrou um novo marcador (tipo_novo_item não é None), salva o item anterior (se tiver texto)
                if item_atual["texto"]:
                     if item_atual["tipo"] == "definicao": definicoes.append(item_atual["texto"])
                     elif item_atual["tipo"] == "exemplo": exemplos.append(item_atual["texto"])
                     elif item_atual["tipo"] == "referencia": referencias.append(item_atual["texto"])

                # Inicia o novo item
                item_atual = {"tipo": tipo_novo_item, "texto": texto_novo_item}


            # Salva o último item acumulado
            if item_atual["texto"]:
                 if item_atual["tipo"] == "definicao": definicoes.append(item_atual["texto"])
                 elif item_atual["tipo"] == "exemplo": exemplos.append(item_atual["texto"])
                 elif item_atual["tipo"] == "referencia": referencias.append(item_atual["texto"])


            # 5. Armazenar dados estruturados (com filtro final)
            if verbete_tupi or definicoes or exemplos or referencias:
                 # Filtra entradas que são apenas marcadores de seção (A, B, C...)
                 # E entradas onde o verbete ficou vazio e não há outros dados
                 if not re.fullmatch(r'[A-Z]\s*$', primeira_linha.strip()) and verbete_tupi:
                    dados_estruturados.append({
                        'id': len(dados_estruturados),
                        'verbete_original_linha': primeira_linha,
                        'verbete_tupi': verbete_tupi,
                        'marcadores': marcadores,
                        'classe_gramatical': classe_gramatical,
                        'definicoes': definicoes,
                        'exemplos': exemplos,
                        'referencias': referencias,
                    })

        except Exception as e:
            print(f"\n!!!!!! Erro Crítico ao processar entrada bruta {i} !!!!!!")
            print(f"Erro: {e}")
            # Limita o tamanho do texto guardado no log de erro
            texto_erro = entrada_bruta[:1000] + "..." if len(entrada_bruta) > 1000 else entrada_bruta
            entradas_com_erro.append({"id_bruta": i, "erro": str(e), "texto": texto_erro})
            # Considere parar o script aqui em caso de erro inesperado, ou continue com cuidado
            # raise e # Descomente para parar o script no primeiro erro grave

    print(f"\nProcessadas {len(dados_estruturados)} entradas estruturadas válidas (v2).")
    print(f"Encontrados {len(entradas_com_erro)} erros durante o processamento.")

    # --- PASSO 6: Filtragem Final (Opcional, mas recomendado) ---
    print("\n--- PASSO 6: Filtrando Entradas Vazias/Inválidas ---")
    dados_filtrados_final = [
        entry for entry in dados_estruturados
        if entry.get('verbete_tupi') and \
           (entry.get('classe_gramatical') or entry.get('definicoes') or entry.get('exemplos') or entry.get('referencias'))
    ]
    num_removidos_filtragem = len(dados_estruturados) - len(dados_filtrados_final)
    print(f"Removidas {num_removidos_filtragem} entradas durante a filtragem final.")
    dados_estruturados = dados_filtrados_final # Usa a lista filtrada

    # --- PASSO 7: Salvar Resultados ---
    print("\n--- PASSO 7: Salvando Resultados ---")
    # ----- Salvar os dados estruturados em JSON -----
    try:
        output_dir_json = os.path.dirname(caminho_arquivo_json)
        if output_dir_json and not os.path.exists(output_dir_json):
            print(f"Criando diretório de saída para JSON: {output_dir_json}")
            os.makedirs(output_dir_json)
        with open(caminho_arquivo_json, 'w', encoding='utf-8') as f_json:
            json.dump(dados_estruturados, f_json, ensure_ascii=False, indent=2)
        print(f"\nDados estruturados finais ({len(dados_estruturados)} entradas) salvos em '{caminho_arquivo_json}'")

        # Mostrar exemplo das 5 primeiras entradas estruturadas
        if dados_estruturados:
            print("\n--- Exemplo das Primeiras 5 Entradas Estruturadas Finais (JSON v2) ---")
            for k in range(min(5, len(dados_estruturados))):
                 print(f"--- Entrada ID {dados_estruturados[k]['id']} ---")
                 print(json.dumps(dados_estruturados[k], ensure_ascii=False, indent=2))
                 print("-" * 20)

    except Exception as e:
        print(f"Erro ao salvar o arquivo JSON final: {e}")

    # ----- Salvar entradas com erro (se houver) -----
    if entradas_com_erro:
        try:
            output_dir_erro = os.path.dirname(caminho_arquivo_erros)
            if output_dir_erro and not os.path.exists(output_dir_erro):
                print(f"Criando diretório de saída para Erros: {output_dir_erro}")
                os.makedirs(output_dir_erro)
            with open(caminho_arquivo_erros, 'w', encoding='utf-8') as f_err:
                 json.dump(entradas_com_erro, f_err, ensure_ascii=False, indent=2) # Salva como JSON para facilitar
            print(f"Informações de {len(entradas_com_erro)} erro(s) salvas em '{caminho_arquivo_erros}'")
        except Exception as e:
            print(f"Erro ao salvar arquivo de erros: {e}")

elif not 'entradas_brutas' in locals() or not entradas_brutas:
     print("Não há entradas brutas para processar. Verifique os Passos 1, 2 e 3.")

print("\n--- Processamento Concluído ---")

--- PASSO 1: Carregando Texto Bruto ---
Texto bruto carregado com sucesso de 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_bruto_extraido.txt' usando encoding 'utf-8'.
Tamanho: 666298 caracteres.

--- PASSO 2: Iniciando Pré-filtragem em Memória ---
Pré-filtragem concluída. 10697 linhas removidas.
Tamanho do texto filtrado: 655031 caracteres.
Texto pré-filtrado (para debug) salvo em 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_pre_filtrado_debug.txt'

--- PASSO 3: Dividindo Texto em Entradas ---
Encontrados 7946 potenciais inícios de entrada no texto filtrado.
Dividido em 7946 entradas brutas.

--- Exemplo das primeiras 3 entradas brutas (Após Divisão) ---
--- Entrada Bruta 1 ---
A
--------------------
--- Entrada Bruta 2 ---
AAN-GATU
--------------------
--- Entrada Bruta 3 ---
A
--------------------

--- PASSO 4: Definindo Função de Limpeza ---
Função de limpeza definida.

--- PASSO 5: Iniciando Processamento e Extração de 7946 Entradas ---

In [22]:
def limpar_texto(texto):
    """Função auxiliar para limpeza básica rápida"""
    if not texto: return ""
    # Remover espaços extras no início/fim e múltiplos espaços internos
    texto = re.sub(r'\s+', ' ', texto).strip()
    # Corrigir erros comuns remanescentes (ADICIONE MAIS AQUI CONFORME NECESSÁRIO)
    texto = texto.replace('Prefi.xo', 'Prefixo')
    texto = texto.replace('Substarítivo', 'Substantivo')
    texto = texto.replace('S)ibsta\'ntivo', 'Substantivo')
    texto = texto.replace('Adj^fcivo', 'Adjetivo')
    texto = texto.replace('Partlculas', 'Partículas')
    texto = texto.replace('Patticulas', 'Partículas')
    texto = texto.replace('Adverbio', 'Advérbio')
    texto = texto.replace('Adverbiorse', 'Advérbio: se') # Exemplo
    texto = texto.replace('náo', 'não')
    texto = texto.replace('Nao', 'Não')
    texto = texto.replace('fungáo', 'função')
    texto = texto.replace('aqáo', 'ação')
    texto = texto.replace('si^ nificado', 'significado') # Corrigir ^
    texto = texto.replace('procedéncia', 'procedência')
    texto = texto.replace('género', 'gênero')
    texto = texto.replace('duplicagao', 'duplicação')
    texto = texto.replace('duplicaqao', 'duplicação')
    texto = texto.replace('multidao', 'multidão')
    texto = texto.replace('ÍABA', 'ABA') # Provável
    texto = texto.replace('f ', '† ') # Talvez usar um símbolo diferente para lusismo/apócope
    texto = texto.replace('+ ', '‡ ') # Talvez usar um símbolo diferente
    # Corrigir numeração inicial das definições
    texto = re.sub(r'^\s*\^', '1.', texto, flags=re.MULTILINE) # ^ -> 1.
    texto = re.sub(r'^\s*z', '2.', texto, flags=re.MULTILINE) # z -> 2.
    # Substituir 9 por ç (com cuidado)
    texto = re.sub(r'\b9\b', 'ç', texto) # 9 isolado
    texto = texto.replace('(t, 9 )', '(t, ç)')
    texto = texto.replace('(Q—,<^—)', '(Q-, ç-)') # Suposição
    texto = texto.replace('(£-)', '(S-)') # Suposição
    return texto

dados_estruturados = []

if 'entradas_brutas' in locals() and entradas_brutas:
    for i, entrada_bruta in enumerate(entradas_brutas):
        entrada_limpa = limpar_texto(entrada_bruta)
        linhas_entrada = entrada_limpa.splitlines()

        if not linhas_entrada:
            continue

        verbete_completo = linhas_entrada[0].strip()
        corpo_texto = "\n".join(linhas_entrada[1:]).strip()

        verbete_tupi = verbete_completo
        marcadores = None
        classe_gramatical = None
        definicoes = []
        exemplos = []
        referencias = []
        notas = []

        # Tentar extrair marcadores do verbete
        match_marcador = re.search(r'(\s*[({\[].*?[)}\]])', verbete_completo)
        if match_marcador:
            marcadores = match_marcador.group(1).strip()
            verbete_tupi = verbete_completo.replace(match_marcador.group(0), '').strip() # Remove marcador do verbete
            verbete_tupi = re.sub(r'[.—]$', '', verbete_tupi).strip() # Remove . ou — do final se houver

        # Tentar extrair classe gramatical (do fim do verbete ou início do corpo)
        padrao_classe = r'([A-Z][a-zA-Zçáéíóúãõü ]+(?: [a-z]+)*)[:.]' # Ex: Substantivo:, Verbo transitivo irregular.
        match_classe_verbete = re.search(padrao_classe + r'\s*$', verbete_tupi) # No fim do verbete limpo
        match_classe_corpo = re.match(padrao_classe, corpo_texto) # No início do corpo

        if match_classe_verbete:
            classe_gramatical = match_classe_verbete.group(1).strip()
            verbete_tupi = verbete_tupi.replace(match_classe_verbete.group(0), '').strip()
        elif match_classe_corpo:
            classe_gramatical = match_classe_corpo.group(1).strip()
            corpo_texto = corpo_texto[match_classe_corpo.end():].strip() # Remove classe do corpo

        # Processar o corpo para definições, exemplos, referências
        # Dividir por linhas novamente pode ajudar a encontrar padrões no início
        linhas_corpo = corpo_texto.splitlines()
        definicao_atual = ""
        for linha in linhas_corpo:
            linha = linha.strip()
            if not linha: continue

            # Verificar exemplos
            match_ex = re.match(r'EX:\s*(.*)', linha, re.IGNORECASE)
            if match_ex:
                if definicao_atual: definicoes.append(definicao_atual.strip())
                definicao_atual = ""
                exemplos.append(match_ex.group(1).strip())
                continue

            # Verificar referências
            match_ref = re.match(r'=\s*(.*)', linha)
            if match_ref:
                if definicao_atual: definicoes.append(definicao_atual.strip())
                definicao_atual = ""
                referencias.append(match_ref.group(1).strip())
                continue

            # Verificar início de nova definição numerada ou com símbolo
            match_num = re.match(r'^([1-9]+\.|[†‡])\s*(.*)', linha)
            if match_num:
                if definicao_atual: definicoes.append(definicao_atual.strip())
                definicao_atual = linha # Começa nova definição
            else:
                # Continuação da definição anterior ou a primeira definição
                definicao_atual += " " + linha

        # Adicionar a última definição acumulada
        if definicao_atual:
            definicoes.append(definicao_atual.strip())

        # Limpar definições (remover números/símbolos iniciais se ainda presentes)
        definicoes_limpas = []
        for d in definicoes:
             d_limpa = re.sub(r'^([1-9]+\.|[†‡])\s*', '', d).strip()
             if d_limpa: definicoes_limpas.append(d_limpa)


        # (Opcional/Avançado: Tentar separar Tupi/Português nos exemplos)

        # Armazenar dados estruturados
        if verbete_tupi or definicoes_limpas or exemplos or referencias: # Só adiciona se tiver algo útil
            dados_estruturados.append({
                'id': i, # Adiciona um ID simples
                'verbete_original': verbete_completo, # Guardar o original pode ser útil
                'verbete_tupi': verbete_tupi,
                'marcadores': marcadores,
                'classe_gramatical': classe_gramatical,
                'definicoes': definicoes_limpas,
                'exemplos': exemplos,
                'referencias': referencias,
                'texto_completo_entrada': entrada_limpa # Guarda o texto completo da entrada para referência
            })

    print(f"\nProcessadas {len(dados_estruturados)} entradas estruturadas.")

    # ----- Salvar os dados estruturados em JSON -----
    try:
        # Garante que o diretório de saída exista
        output_dir_json = os.path.dirname(caminho_arquivo_json)
        if not os.path.exists(output_dir_json):
            os.makedirs(output_dir_json)

        with open(caminho_arquivo_json, 'w', encoding='utf-8') as f_json:
            # indent=2 torna o arquivo JSON legível por humanos
            json.dump(dados_estruturados, f_json, ensure_ascii=False, indent=2)
        print(f"\nDados estruturados salvos em '{caminho_arquivo_json}'")

        # Mostrar exemplo da primeira entrada estruturada
        if dados_estruturados:
            print("\n--- Exemplo da Primeira Entrada Estruturada (JSON) ---")
            print(json.dumps(dados_estruturados[0], ensure_ascii=False, indent=2))

    except Exception as e:
        print(f"Erro ao salvar o arquivo JSON: {e}")

elif 'entradas_brutas' in locals() and not entradas_brutas:
     print("A divisão em entradas brutas falhou ou não gerou resultados.")
else:
     print("Execute os passos anteriores primeiro.")


Processadas 1485 entradas estruturadas.

Dados estruturados salvos em 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dicionario_estruturado.json'

--- Exemplo da Primeira Entrada Estruturada (JSON) ---
{
  "id": 0,
  "verbete_original": "A",
  "verbete_tupi": "A",
  "marcadores": null,
  "classe_gramatical": null,
  "definicoes": [],
  "exemplos": [],
  "referencias": [],
  "texto_completo_entrada": "A"
}


### Validação e Refinamento do JSON

In [2]:
import json
import pandas as pd
import random

caminho_arquivo_json = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dicionario_estruturado_final.json'
try:
    with open(caminho_arquivo_json, 'r', encoding='utf-8') as f:
        dados_estruturados = json.load(f)
    print(f"JSON carregado com {len(dados_estruturados)} entradas.")

    # Opcional: Converter para DataFrame Pandas para fácil visualização/filtragem
    df = pd.DataFrame(dados_estruturados)
    pd.set_option('display.max_rows', 100) # Mostrar mais linhas
    pd.set_option('display.max_colwidth', 200) # Mostrar mais texto nas colunas

    # Mostrar algumas entradas aleatórias
    print("\n--- Amostra Aleatória de Entradas (DataFrame Pandas) ---")
    print(df.sample(n=15)) # Veja 15 entradas aleatórias

    # Mostrar as primeiras e últimas entradas também pode ser útil
    # print("\n--- Primeiras 5 Entradas ---")
    # print(df.head(5))
    # print("\n--- Últimas 5 Entradas ---")
    # print(df.tail(5))

except FileNotFoundError:
    print(f"Erro: Arquivo JSON não encontrado em '{caminho_arquivo_json}'")
    dados_estruturados = None
    df = None
except json.JSONDecodeError as e:
    print(f"Erro ao decodificar o JSON: {e}")
    dados_estruturados = None
    df = None
except Exception as e:
     print(f"Erro inesperado ao carregar/processar JSON: {e}")
     dados_estruturados = None
     df = None

JSON carregado com 7922 entradas.

--- Amostra Aleatória de Entradas (DataFrame Pandas) ---
        id  \
4530  4530   
7671  7671   
6950  6950   
1933  1933   
5139  5139   
1991  1991   
7651  7651   
5879  5879   
5968  5968   
2688  2688   
1787  1787   
4811  4811   
3521  3521   
7125  7125   
4744  4744   

                                                                                     verbete_original_linha  \
4530                             MO-IO-BÃI-x0ARqA. Verbo transitivo: por um diante do outro. - mo-Io-bãi-a.   
7671                                                   YBYT-I-MBORA. Substantivo: pó, poeira. = ybytu-byra.   
6950                                                                  TAPERÃ. Ornitologia: andorinha grande   
1933                                                                EK0-PORANGA (T-). Substantivo: virtude,   
5139                       NDOaRA. Participio sufixo: o que costuma ser ou estar. S na- salacão de "guara".   
1991             

### Geração do Dataset

In [4]:
import json

caminho_arquivo_json = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dicionario_estruturado_final.json'
caminho_dataset_txt = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dataset_tupi.txt'
dados_estruturados = []

try:
    with open(caminho_arquivo_json, 'r', encoding='utf-8') as f:
        dados_estruturados = json.load(f)
    print(f"JSON carregado com {len(dados_estruturados)} entradas para gerar dataset TXT.")

    with open(caminho_dataset_txt, 'w', encoding='utf-8') as f_out:
        count_linhas = 0
        for entrada in dados_estruturados:
            verbete = entrada.get('verbete_tupi')
            exemplos_tupi = [] # Você precisaria implementar a extração dos exemplos em Tupi

            # Escrever verbete (se válido)
            if verbete and isinstance(verbete, str) and verbete.strip():
                 # Pode adicionar filtros aqui (ex: ignorar verbetes muito curtos ou com caracteres estranhos)
                 if len(verbete.split()) < 10: # Heurística simples para evitar definições coladas
                    f_out.write(verbete.strip() + '\n')
                    count_linhas += 1

            # Escrever exemplos em Tupi (se houver)
            # for ex_tupi in exemplos_tupi:
            #      if ex_tupi and isinstance(ex_tupi, str) and ex_tupi.strip():
            #           f_out.write(ex_tupi.strip() + '\n')
            #           count_linhas += 1

    print(f"Dataset de texto Tupi gerado em '{caminho_dataset_txt}' com {count_linhas} linhas.")

except Exception as e:
    print(f"Erro ao gerar dataset TXT: {e}")

JSON carregado com 7922 entradas para gerar dataset TXT.
Dataset de texto Tupi gerado em 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dataset_tupi.txt' com 4961 linhas.
